<a href="https://colab.research.google.com/github/jafr0822/MNA_ProyectoIntegrador-Equipo7/blob/main/Avance4_7_Equipo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Proyecto Integrador


## Tecnológico de Monterrey
### Maestría en Inteligencia Artificial Aplicada (MNA)
#### Avance 4
#### Equipo 7


* Jorge Arturo Federico Rivera – A01250724
* Marco Antonio Vázquez Morales – A01793704
* Alejandro Jesús Vázquez Navarro - A01793146

### Modelos Alternativos

Proyecto:

*Modelo clasificador de multimorbilidad maternal y predictor de desenlaces perinatales a partir de datos clínicos metabólicos, genéticos y nutricionales de mujeres mexicanas*

23 de mayo de 2024

# **1. Carga de Librerías**

In [2]:
#%pip install imbalanced-learn

import pandas as pd
from sklearn.model_selection import train_test_split

# **2. Carga de datos**


In [8]:
file_path = r'data/data_limpia.csv'

#Eliminamos columnas que no necesitaremos como identificarores y el número de clúster

cols_to_remove = ['id_gdg', 'origen_px', 'IndexMorbilidad', 'anticonceptivo_0.0',
       'anticonceptivo_0.6316526610644257', 'anticonceptivo_1.0',
       'anticonceptivo_2.0', 'cluster']

data = pd.read_csv(file_path, sep=";", encoding='utf-8')
data  = data.drop(cols_to_remove , axis=1)

data.head()

,EscalaRiesgo,biopsias,obs_h,ichos_pregesta,hta_pregesta,sop,hipotiroidismo,hipertiroidismo,consumo_alcohol,consumo_tabaco,...,sdg_parto,ant_aborto,macrosomia_rn_0,macrosomia_rn_0.0,macrosomia_rn_0/0,macrosomia_rn_1,macrosomia_rn_1.0,macrosomia_rn_1/0,macrosomia_rn_1/1,macrosomia_rn_2
0,C,0,0,0,0,0,0,0,0,0,...,0.0,-0.747297,0,0,0,1,0,0,0,0
1,C,0,0,0,1,0,0,0,0,0,...,0.0,-0.747297,0,0,0,1,0,0,0,0
2,C,0,1,0,1,0,0,0,0,0,...,0.0,-0.747297,0,0,0,1,0,0,0,0
3,B,0,0,0,1,0,0,0,0,0,...,0.0,-0.747297,0,0,0,1,0,0,0,0
4,C,0,1,0,1,0,0,0,0,0,...,0.0,1.420203,0,0,0,1,0,0,0,0


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

X = data.drop('EscalaRiesgo', axis=1)
y = data['EscalaRiesgo']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# **3. Aplicar balanceo de clases via SMOTE**


Esta técnica es recomendada cuando tenemos clases desbalanceadas. Nuestro problema tiene 3 clases:

- A Para alto riesgo (la presencia de 3 o 4 factores multimorbilidad)
- B Para medio riesgo (presencia de 1 o 2 factores multimorbilidad)
- C Para riesgo nulo. (sin la presencia de factores de multimorbilidad)

En temas de salud es importante tener clases balanceadas, esto nos permitirá tener los siguientes beneficios:

- Mejora en la precisión del modelo; en este sentido, detectar una clase **A** con alto riesgo de morbilidad es crucial para que el modelo entregue valor al área médica.

- Equidad en la atención médico. Debemos aseguramos que nuestro modelo sea justo y equitativo sobre todo en temas de salud. Un desbalance puede provocar sesgos y lecturas erróneas.

- Generalización. Un modelo con clases balanceadas generalizará mejor cuando trate nuevos datos. En el dominio de conocimiento de salud humana, esto es relevante.

In [10]:
from imblearn.over_sampling import SMOTE


smote = SMOTE(random_state=42)


X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Check the class distribution
print("Original class distribution:")
print(y_train.value_counts())
print("\nBalanced class distribution:")
print(y_train_balanced.value_counts())

Original class distribution:
EscalaRiesgo
C    645
B    305
A      9
Name: count, dtype: int64

Balanced class distribution:
EscalaRiesgo
C    645
B    645
A    645
Name: count, dtype: int64


2.